##### Copyright 2018 The TensorFlow Authors.

## Setup

In [10]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow as tf
import pandas as pd
import numpy as np
import pandas as pd
import re   
import string
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.utils import to_categorical
tfds.disable_progress_bar()

Import `matplotlib` and create a helper function to plot graphs:

In [11]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [12]:
# ## Method for getting input
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# nltk.download("punkt")
# nltk.download('wordnet')
# nltk.download("stopwords")

# def parse_text(text):

#     ## Tokenize string into words (and punctuation)
#     word_array = word_tokenize(text)
#     word_array = [word.lower() for word in word_array if word.isalpha()]

#     ## Filter out stop words
#     stop_words = set(stopwords.words("english"))
#     filtered_words = [word for word in word_array if word.casefold() not in stop_words]

#     ## Turn words into lemmatized words
#     lemmatizer = WordNetLemmatizer()
#     lemitized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

#     ## Apply Stemming (Find the roots of similar words)
#     stemmer = PorterStemmer()
#     stemmed_words = [stemmer.stem(word) for word in lemitized_words]

#     joined_words = ""
#     #for word in stemmed_words:
#     joined_words = " ".join(list(stemmed_words))
#     return joined_words

In [13]:
###makes clean file

# import csv
# csv.field_size_limit(10000000)
# text_list = []
# with open("master_dataset.csv", newline='') as f:
#     #reader = csv.reader(f)
#     data = list(csv.reader(f))
# #print(data[:10])
# for row in data:
    
#     #print(parse_text(row[2]))
#     text_list.append(str(parse_text(row[2])))

# #print(text_list[:10])
# with open("master_dataset_clean.csv", 'w') as f2:
#     #writer = csv.writer(f2)
#     for x in text_list:
#         f2.write(x)
#         f2.write('\n')
        #writer.writerows([row])
            

In [14]:
tickets = pd.read_csv("master_dataset_clean.csv", skip_blank_lines = False, keep_default_na=False)
tickets = tickets.fillna('[UNK]')
tickets2 = pd.read_csv("labelled_dataset.csv")
tickets3 = pd.DataFrame()

#print(tickets['desc'][:3])
#tickets['desc'] = [parse_text(text) for text in tickets['desc']]
#tickets['desc'] = tickets['desc'].apply(parse_text())
#print(tickets['desc'][:3])

text_column = tickets['desc']
tickets3 = pd.concat([tickets3,text_column], axis = 1)
label_column = tickets2['Grade'].astype(int)
label_column = label_column - 1
tickets3 = pd.concat([tickets3,label_column], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(text_column, label_column, test_size=0.2, shuffle=True)

y_train_cat = to_categorical(y_train, 3)
y_test_cat = to_categorical(y_test, 3)


X_train_dataset = tf.data.Dataset.from_tensor_slices(X_train)
X_test_dataset = tf.data.Dataset.from_tensor_slices(X_test)
# y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train)
# y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test)
y_train_dataset = tf.data.Dataset.from_tensor_slices(y_train_cat)
y_test_dataset = tf.data.Dataset.from_tensor_slices(y_test_cat)

test_dataset = tf.data.Dataset.zip((X_test_dataset, y_test_dataset))
train_dataset = tf.data.Dataset.zip((X_train_dataset, y_train_dataset))

# print(train_dataset.element_spec)
# print(test_dataset.element_spec)

In [15]:
np.array(train_dataset)

array(<_ZipDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(3,), dtype=tf.float32, name=None))>,
      dtype=object)

In [16]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'r support logic vector'
label:  [0. 1. 0.]


Next shuffle the data for training and create batches of these `(text, label)` pairs:

In [17]:
BUFFER_SIZE = 10000 #10000
BATCH_SIZE = 512 #64

In [18]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [19]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'miss file sampl'
 b'includ number termtyp entri correspond incoterm http came effect januari http version incoterm remain valid long parti agre version incoterm list partyseeddata date also appear date version ticket updat partyseeddata includ incoterm exist incoterm maintain current id parent term descript alter indic incoterm version refer'
 b'server send host level paramet updat finish right start execut command gener may led perform autostart command agent side execut start command']

labels:  [[1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


## Create the text encoder

The raw text loaded by `tfds` needs to be processed before it can be used in a model. The simplest way to process text for training is using the `TextVectorization` layer. This layer has many capabilities, but this tutorial sticks to the default behavior.

Create the layer, and pass the dataset's text to the layer's `.adapt` method:

In [20]:
VOCAB_SIZE = 2500 #1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

The `.adapt` method sets the layer's vocabulary. Here are the first 20 tokens. After the padding and unknown tokens they're sorted by frequency:

In [21]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'code', 'use', 'file', 'error', 'http', 'test',
       'line', 'method', 'class', 'info', 'run', 'new', 'creat', 'java',
       'user', 'fail', 'set', 'add'], dtype='<U30')

Once the vocabulary is set, the layer can encode text into indices. The tensors of indices are 0-padded to the longest sequence in the batch (unless you set a fixed `output_sequence_length`):

In [22]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[221,   4, 828, ...,   0,   0,   0],
       [102, 118,   1, ...,   0,   0,   0],
       [ 72, 270, 320, ...,   0,   0,   0]])

In [23]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'miss file sampl'
Round-trip:  miss file sampl                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

Original:  b'includ number termtyp entri correspond incoterm http came effect jan

## Create the model

Above is a diagram of the model.

1. This model can be build as a `tf.keras.Sequential`.

2. The first layer is the `encoder`, which converts the text to a sequence of token indices.

3. After the encoder is an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices to sequences of vectors. These vectors are trainable. After training (on enough data), words with similar meanings often have similar vectors.

  This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a `tf.keras.layers.Dense` layer.

4. A recurrent neural network (RNN) processes sequence input by iterating through the elements. RNNs pass the outputs from one timestep to their input on the next timestep.

  The `tf.keras.layers.Bidirectional` wrapper can also be used with an RNN layer. This propagates the input forward and backwards through the RNN layer and then concatenates the final output.

  * The main advantage of a bidirectional RNN is that the signal from the beginning of the input doesn't need to be processed all the way through every timestep to affect the output.  

  * The main disadvantage of a bidirectional RNN is that you can't efficiently stream predictions as words are being added to the end.

5. After the RNN has converted the sequence to a single vector the two `layers.Dense` do some final processing, and convert from this vector representation to a single logit as the classification output.


The code to implement this is below:

In [24]:
# model = tf.keras.Sequential([
#     encoder,
#     tf.keras.layers.Embedding(
#         input_dim=len(encoder.get_vocabulary()),
#         output_dim=64,
#         # Use masking to handle the variable sequence lengths
#         mask_zero=True),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     #tf.keras.layers.Dense(1)
#     tf.keras.layers.Dense(3, activation='softmax')
# ])


model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

Please note that Keras sequential model is used here since all the layers in the model only have single input and produce single output. In case you want to use stateful RNN layer, you might want to build your model with Keras functional API or model subclassing so that you can retrieve and reuse the RNN layer states. Please check [Keras RNN guide](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse) for more details.

The embedding layer [uses masking](https://www.tensorflow.org/guide/keras/masking_and_padding) to handle the varying sequence-lengths. All the layers after the `Embedding` support masking:

In [25]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True, True, True]


To confirm that this works as expected, evaluate a sentence twice. First, alone so there's no padding to mask:

In [26]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

2023-07-21 13:51:56.254311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8903


1/1 [==============================] - 5s 5s/step
[0.332143   0.33191156 0.3359454 ]


2023-07-21 13:51:56.544692: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Now, evaluate it again in a batch with a longer sentence. The result should be identical:

In [27]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

1/1 [==============================] - 0s 263ms/step
[0.332143   0.33191156 0.3359454 ]
1/1 [==============================] - 0s 263ms/step
[0.332143   0.33191156 0.3359454 ]


Compile the Keras model to configure the training process:

In [28]:
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               optimizer=tf.keras.optimizers.Adam(1e-4),
#               metrics=['accuracy'])

model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=False),
    #optimizer=tf.keras.optimizers.SGD(),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=["accuracy"] #tf.keras.metrics.CategoricalAccuracy()
)

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (Text  (None, None)              0         
 Vectorization)                                                  
                                                                 
 embedding (Embedding)       (None, None, 32)          80000     
                                                                 
 bidirectional (Bidirection  (None, None, 128)         49664     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                41216     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

## Train the model

In [30]:
#find weights for labels

samp0 = 54046
samp1 = 24077
samp2 = 9863
tot_samples = samp0 + samp1 + samp2

w0 = tot_samples / (3*samp0)
w1 = tot_samples / (3*samp1)
w2 = tot_samples / (3*samp2)

In [31]:
history = model.fit(train_dataset, 
                    epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30,
                    #class_weight = {0:w0,1:w1,2:w2},
                    #use_multiprocessing = True,
                    #workers = 5
                    )
#class weight has made it worse

Epoch 1/10


2023-07-21 13:52:06.582951: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_42/output/_24'
2023-07-21 13:52:10.619021: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f16901a6f90 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-21 13:52:10.619052: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, 

 69/138 [==============>...............] - ETA: 10:01 - loss: 1.0808 - accuracy: 0.5221

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))

In [ ]:
predictions

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was not good. The animation and the graphics '
               'were terrible. I would not recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)

print(np.argsmax(predictions[0]))